# Scenario 3: Multiple data scientist working on multiple ML models

MLflow setup:

- Tracking server: yes, remote server (EC2).
- Backend store: postgresql database.
- Artifacts store: s3 bucket.

Experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example, you'll need an AWS AWS account.

In [7]:
import os
os.environ["AWS_PROFILE"] = "diogofn1"

In [9]:
import mlflow
import os

# os.environ['AWS_PROFILE']

TRACKING_SERVER_HOST = "ec2-52-54-140-71.compute-1.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")


In [2]:
print(f"tracking URI: {mlflow.get_tracking_uri()}")

tracking URI: http://ec2-52-54-140-71.compute-1.amazonaws.com:5000


In [10]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifact-remote-diogo/1', creation_time=1761576016006, experiment_id='1', last_update_time=1761576016006, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifact-remote-diogo/0', creation_time=1761575315898, experiment_id='0', last_update_time=1761575315898, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, name="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/27 14:57:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifact-remote-diogo/1/042b473336ed4d3a8f35f47a2a3d061d/artifacts'
🏃 View run amusing-fox-701 at: http://ec2-52-54-140-71.compute-1.amazonaws.com:5000/#/experiments/1/runs/042b473336ed4d3a8f35f47a2a3d061d
🧪 View experiment at: http://ec2-52-54-140-71.compute-1.amazonaws.com:5000/#/experiments/1


In [12]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifact-remote-diogo/1', creation_time=1761576016006, experiment_id='1', last_update_time=1761576016006, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifact-remote-diogo/0', creation_time=1761575315898, experiment_id='0', last_update_time=1761575315898, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [13]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [14]:
client.search_registered_models()

[]

In [15]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

mlflow.register_model(
    model_uri=f"run:/{run_id}/models",
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2025/10/27 15:09:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1761577785308, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761577785308, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='', run_link='', source='run:/042b473336ed4d3a8f35f47a2a3d061d/models', status='READY', status_message=None, tags={}, user_id='', version='1'>